# Table of content
- Task
- Draft
- Aciton
    - 0 封装指令功能对应函数
    - 1 天气查询相关
        - 1.1 判断是否为城市名
        - 1.2 学习使用心知天气 API 
        - 1.3 学习使用 JSON 模块
        - 1.4 解析JSON数据并输出天气信息
    - 2 主函数
- Advanced features
    - 0 温度单位转换
    - 1 指定日期查询天气
    - 2 输入自动补全
    - 3 从嵌套的 JSON 数据中优雅得取数据
- Timelog

## Task
完成一个网络版的天气查询程序，运行在命令行界面，实现以下功能：

- 输入城市名，返回该城市的天气数据；
- 输入指令，打印帮助文档（一般使用 h 或 help）；
- 输入指令，退出程序的交互（一般使用 q 或 quit）；




因为是网络版，本章需要了解API知识，从而也需要了解JSON
- API 是什么
服务商提供的一个数据通信接口，你向它发起一定格式的请求，它返回一定格式的数据
- API 怎么用
对提供的API URL 使用 get 方法，带query参数

- JSON 是什么
在查询详细资料之前，我了解过也使用过 JSON，我的理解是：它是按照一定格式存储的数据，是网络上服务端与客户端数据交换时的载体，它的产生我觉得有两个原因：
- 因为网络传输时只能传输文本信息，无法按照数据的数据类型直接传输，比如stu_list=['a', 'b', 'b'], 网络无法直接传输该数据，只能变成字符串传输，接收端接收到以后，即使知道它是list类型，也不能把它直接变成list类型的数据
- 一个项目的不同部分可能由不同语言写成，为了不同部分之间能够方便的进行数据交换，需要一种格式的数据，不同语言接受到它时都能转化为自己可识别类型的数据
JSON就可以解决这两个问题。

- JSON 怎么用
一般的语言都有自己的 JSON 模块，可以完成 数组/字典类型数据与JSON格式字符串的互相转换
python demo见下方代码

In [27]:
import json
dt = {
    'a':1
}
data_for_trans = json.dumps(dt) # string
print('{!r}, type:{}'.format(data_for_trans, type(data_for_trans)))

data_in_python = json.loads(data_for_trans) # dict
print('{!r}, type:{}'.format(data_in_python, type(data_in_python)))

'{"a": 1}', type:<class 'str'>
{'a': 1}, type:<class 'dict'>


## Draft

- 获得用户合法输入
    - while-loop + try/except + input() 

- 根据输入判断并作出反应
    - 指令
        - 封装功能至函数
            - 'h', 'help' : 不封装, 直接print(help_doc)即可
            - 'history'：print_history()
            - 'q, 'quit': quit_()
            
        - 判断是指令
            - if user_input in ['h', 'help']
        - 调用对应函数
            - print(help_doc)
            - print_history(history)
            - quit_()
            
    - 城市名
        - 判断是城市名
            - is_city()
        - 作出响应
            - 将用户输入的城市名加入 API 查询时所用的参数字典中
            - 调用 API 查询天气 
            - 解析 API 返回的数据并 print 天气信息
            - 保存查询记录

- 主体函数 main()
    - 获取用户输入
    - 判断输入，然后调用不同函数

# Action
## 0  封装指令功能至函数

In [6]:
from textwrap import dedent

# dedent 能够在输出以下这种多行字符串时，去掉每行前面的空格
help_doc = dedent("""\
    ----天气资料由「心知天气」提供----
    - 输入城市名，返回该城市的天气数据；
    - 输入指令 h 或 help，打印帮助文档；
    - 输入指令 q 或 quit，退出本程序；
    - 输入指令 c 或 C，将温度单位设定为摄氏度, 程序启动后默认单位为摄氏度；
    - 输入指令 f 或 F，将温度单位设定为华氏度；
    - 输入指令 now，进入查询实时天气模式， 程序启动后默认为该模式；
    - 输入指令 daily，进入查询单日/多日天气概况模式；
    - 输入指令 history，打印查询过的所有城市。
    """)


def print_history(his_deque):
    """
    his_deque
    :param his_deque: (deque), 存放用户天气查询历史
    :return: None
    """
    if len(his_deque) == 0:
        print('您还没有查询过天气')
    else:
        print('您的天气查询历史记录是：')
        for i in his_deque:
            print(i)


def quit_(his_deque):
    """输出历史记录，然后退出程序"""
    print_history(his_deque)
    print("正在退出程序...")
    exit(0)

## 1 天气查询相关

### 1.1 判断是否为城市名
为了减少因不判断而无效调用API的次数，每天API调用次数有限，需进行初步预判。因调用心知天气API合法城市名中文和拼音都可以，所以
思路：如果全部是中文或拼音，则返回True

In [1]:
from string import ascii_letters

def is_pinyin(user_input):
    """
        检测字符是否只含有拼音

        :param user_input:(str), 需要判断的字符串
        :return: 字符串全部为拼音时返回 True；否认返回False
        """

    for i in user_input:
        # 使用中文十六进制的 Unicode 字符集范围判断
        if i not in ascii_letters:
            return False

    return True


def is_chinese(user_input):
    """
    检测字符是否只含有中文

    :param user_input:(str), 需要判断的字符串
    :return: 字符串全部为中文时返回 True；否认返回False
    """

    for i in user_input:
        # 使用中文十六进制的 Unicode 字符集范围判断
        if i < '\u4e00' or i > '\u9fa5':
            return False

    return True


def is_city(user_input):
    """
    通过检测字符是否只含有中文，或只含有拼音，粗略判断是否为城市名
    减少无效调用 API 的次数
    :param user_input:(str), 需要判断的字符串
    :return: 字符串全部为中文, 或者全部为拼音时返回 True；否认返回False
    """
    if is_pinyin(user_input) or is_chinese(user_input):
        return True
    else:
        return False
    
# 测试
print(is_city('北京'))
print(is_city('beijing'))
print(is_city('!'))
print(is_city('北京aaa'))


True
True
False
False


### 1.2 学习使用心知天气 API 
当时课堂提供的国内API有两个，比较了下文档我选择了心知API，因为它有Python调用API 的 demo，可以迅速上手。
心知API给了两个 demo，分别是使用 requests库 和 urllib 库 调用 API，因为之前使用过 requests库，所以直接上手学该demo。
初步的代码见下方，基本分为

In [4]:
import requests

def fetchWeather(location):
    """
    调用心知天气查询 API
    :param location: 查询城市的汉语拼音
    :return: json 格式的数据
    """

    result = requests.get('https://api.seniverse.com/v3/weather/now.json', params={
        'key': '4r9bergjetiv1tsd',
        'location': location,
        'language': 'zh-Hans',
        'unit': 'c'
    }, timeout=10)

    return result.text


location = 'NANYANG'
result_json = fetchWeather(location)
print(result_json)

{"results":[{"location":{"id":"WTB58RQWFJC5","name":"南阳","country":"CN","path":"南阳,南阳,河南,中国","timezone":"Asia/Shanghai","timezone_offset":"+08:00"},"now":{"text":"多云","code":"4","temperature":"26","feels_like":"26","pressure":"991","humidity":"57","visibility":"10.0","wind_direction":"东北","wind_direction_degree":"57","wind_speed":"12.96","wind_scale":"3","clouds":"","dew_point":""},"last_update":"2017-08-26T20:40:00+08:00"}]}


如果给出的城市不存在，会得到什么的信息呢，知道后可以帮助我们处理异常，让代码更健壮

简单观察发现，返回的数据如果转换为 dict，没有了"result"这个key，多出了"status"这个key，我们可以依此特征作出判断返回信息是否正常。

### 1.3 学习使用 JSON 模块
有上面可见，返回了这么一大长串的数据，为了能把它转换为 Python 内部的数据结构，同时方便的看明白它的嵌套结构，搜索了后学习了 JSON 模块的两个用法

In [5]:
import json

# 字符串格式的 json 数据 ->  python 内的数据结构
dt = json.loads(result_json)

#  python 内的数据结构 -> 字符串格式的 json 数据
# 并且带缩进和和换行，方便观看嵌套结构
result_goodstyle = json.dumps(dt, indent=2, ensure_ascii=False)
print(result_goodstyle)

{
  "results": [
    {
      "location": {
        "id": "WTB58RQWFJC5",
        "name": "南阳",
        "country": "CN",
        "path": "南阳,南阳,河南,中国",
        "timezone": "Asia/Shanghai",
        "timezone_offset": "+08:00"
      },
      "now": {
        "text": "多云",
        "code": "4",
        "temperature": "26",
        "feels_like": "26",
        "pressure": "991",
        "humidity": "57",
        "visibility": "10.0",
        "wind_direction": "东北",
        "wind_direction_degree": "57",
        "wind_speed": "12.96",
        "wind_scale": "3",
        "clouds": "",
        "dew_point": ""
      },
      "last_update": "2017-08-26T20:40:00+08:00"
    }
  ]
}


### 1.4 解析JSON数据并输出天气信息
学会了调用API和JSON的两个简单用法，并且有了基本的异常判断条件，就可以开始解析API返回的数据，取出我们需要的数据，组成最终输出给用户的天气信息。接下来，就开始写这个函数

In [15]:
def parse_json(weather_data):
    """
    解析心知天气API返回的json格式数据，返回固定格式的天气信息
    :param weather_data: (str), 天气 API 返回的 json 格式数据
    :return:(str), 当 API 返回正常时，返回天气信息；
             (None), 当 API 返回错误时，返回 None
    """
    # 解析 json 数据，获得 dict 格式数据
    result_dict_from_api = json.loads(weather_data)

    # 判断API 返回数据是否正常
    # 如果正常，返回定制的天气信息
    if result_dict_from_api.get('results', None) is not None:
        # 将数据拆分为 3 部分
        # location_dict: 存有天气所在地理位置信息的字典
        # details_dict: 存有该地理位置的详细天气信息的字典，包括晴雨/温度等
        # updated_time_raw: 所获天气信息的更新时间
        location_dict = result_dict_from_api['results'][0]['location']
        details_dict = result_dict_from_api['results'][0]['now']
        updated_time_raw = result_dict_from_api['results'][0]['last_update']

        # 定制输出需要的天气信息
        updated_time_print = updated_time_raw.replace('T', ' ').replace('+08:00', '')
        result_for_user = f"""\
        {location_dict['name']}的天气为{details_dict['text']}
        气温: {details_dict['temperature']}摄氏度
        风向：{details_dict['wind_direction']}
        更新时间:{updated_time_print}
        """
        print(result_for_user)
        return result_for_user

    # 如果返回 json 数据内有 status，则异常
    elif result_dict_from_api.get('status', None):
        print('API返回异常,异常信息:', result_dict_from_api.get('status'))
        return None
    else:
        print('API返回异常，异常信息:', result_dict_from_api)
        return None

# 测试函数
location = 'beijing'
result_json = fetchWeather(location)
parse_json(result_json)

        北京的天气为多云
        气温: 26摄氏度
        风向：西南
        更新时间:2017-08-26 17:50:00
        


'        北京的天气为多云\n        气温: 26摄氏度\n        风向：西南\n        更新时间:2017-08-26 17:50:00\n        '

## 2 主函数

最麻烦的解析函数这一关也解决了，我们的程序也就基本上写好了，把以上几部分组在一起，写出主函数main()，就得到ch2 天气查询网络版的 MVP 版本


In [22]:
from collections import deque
# 使用 collections 模块下的 deque 数据结构，其类似list，但可以更快的完成插入删除

# 程序主函数
def main():
    # api 查询时需要的参数
    api_params_dict = dict(
    key = '4r9bergjetiv1tsd', # API key
    api = 'https://api.seniverse.com/v3/weather/now.json',  # API URL，可替换为其他 URL
    unit = 'c',  # 单位
    language = 'zh-Hans',  # 查询结果的返回语言
    )
    # 用来保存历史记录
    history_deque = deque()

    while True:
        try:
            user_input = input('请输入城市名称或指令，输入h/help获取帮助信息:')
        except EOFError:
            print('您选择了退出交互。')
            break
        except:
            print('未知错误')
            break

        else:
            if user_input in ['h', 'help']:
                print_help()
            elif user_input in ['history']:
                print_history(history_deque)
            elif user_input in ['q', 'quit']:
                quit_(history_deque)
            elif iscity(user_input):
                # 调用 API 查询天气，获取 json 数据
                weather_json_data = fetch_weather(user_input, **api_params_dict)
                # 解析 API 返回的 json 数据，生成定制天气信息或打印异常信息
                weather_search_result = parse_json(weather_json_data)
                if weather_search_result is not None:
                    history_deque.append(weather_search_result)
                else:
                    continue
            else:
                print('输入非法')


if __name__ == '__main__':
    main()


请输入城市名称或指令，输入h/help获取帮助信息:beijing、
输入非法
请输入城市名称或指令，输入h/help获取帮助信息:beijing
        北京的天气为多云
        气温: 26摄氏度
        风向：西南
        更新时间:2017-08-26 18:10:00
        
请输入城市名称或指令，输入h/help获取帮助信息:q
您的天气查询历史记录是：
        北京的天气为多云
        气温: 26摄氏度
        风向：西南
        更新时间:2017-08-26 18:10:00
        
正在退出程序...


SystemExit: 0

C:\Anaconda2\envs\py36\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## 进阶功能


### 0 温度转换
因为看API文档时发现，心知天气根据调用 API 时的 unit='c' or unit='f' 分别提供摄氏度和华氏度的数据，所以构思功能实现思路：
- 设定一个变量，标识当前温度单位是'c' 还是 'f'
- 通过输入，用户可以更改当前温度单位对应的变量值
    - 当更改生效后，程序输出当前的温度单位，让用户可以感知改变
- 调用 API 时，更新 API 参数字典的 `unit`对应的 `value`

代码见下方

In [ ]:
# 标识当前温度单位的变量
# 变量的值取 api_params_dict 中 key unit 对应的 value： 'c' / 'f'
temperature_unit = 'c'  # 默认温度单位为摄氏度

# 使用 requests 调用 api 时需要用的参数
api = 'https://api.seniverse.com/v3/weather/daily.json',
default_api_params_dict = dict(
    key = '4r9bergjetiv1tsd', # API key
    unit = 'c',  # 单位 c 是摄氏度，f 是 华氏度
    language = 'zh-Hans',  # 查询结果的返回语言
)

# 更改生效后的提示函数
def temperature_switch_prompt(temperature_unit):
    """
    根据用户输入的温度单位，输出对应的交互语句
    :param temperature_unit: 'c','C', 'f', 'F'
    :return: None
    """
    if temperature_unit in ['c', 'C']:
        print('您已经选择温度显示单位为:摄氏度')
    else:
        print('您已经选择温度显示单位为:华氏度')
        
# 测试新功能
while 1:
    user_input = input('指令改变温度显示单位')
    if user_input in ['c', 'f']:
        temperature_unit = user_input
        temperature_switch_prompt(temperature_unit)
    else:
        break
        

请输入城市名称或指令，输入h/help获取帮助信息:c
您已经选择温度显示单位为:摄氏度
请输入城市名称或指令，输入h/help获取帮助信息:f
您已经选择温度显示单位为:华氏度


上面的简单小测试通过后，我们就可以在用户查询天气时，把 api_params_dict 对应的'unit'根据当前模式进行更新，然后再调用API
此时还需要做两件事：
- 改变 api_params_dict 后，返回的 json 数据格式会变吗？如果有变化，我们也需要做对应修改
- 输出给用户的文本也要做相应的修改，不然即使获取了华氏度的结果，输出给用户的还是  气温：xx摄氏度

In [6]:
"""
改变 api_params_dict 后，对比返回的 json 数据格式
"""
def fetchWeather(location, temperature_unit='c'):
    """
    调用心知天气查询 API
    :param location: 查询城市的汉语拼音
    :return: json 格式的数据
    """

    result = requests.get('https://api.seniverse.com/v3/weather/now.json', params={
        'key': '4r9bergjetiv1tsd',
        'location': location,
        'language': 'zh-Hans',
        'unit': temperature_unit
    }, timeout=10)

    return result.text

def format_json(json_data):
    dt = json.loads(json_data)
    format_data = json.dumps(dt, indent=2, ensure_ascii=False)
    return format_data


location = 'beijing'
# 使用摄氏度
result_c = fetchWeather(location, temperature_unit='c')

# 使用华氏度
result_f = fetchWeather(location, temperature_unit='f')
# 对比结果
print(format_json(result_c))
print(format_json(result_f))

{
  "results": [
    {
      "location": {
        "id": "WX4FBXXFKE4F",
        "name": "北京",
        "country": "CN",
        "path": "北京,北京,中国",
        "timezone": "Asia/Shanghai",
        "timezone_offset": "+08:00"
      },
      "now": {
        "text": "阴",
        "code": "9",
        "temperature": "24",
        "feels_like": "24",
        "pressure": "1011",
        "humidity": "61",
        "visibility": "14.8",
        "wind_direction": "西",
        "wind_direction_degree": "248",
        "wind_speed": "12.96",
        "wind_scale": "3",
        "clouds": "",
        "dew_point": ""
      },
      "last_update": "2017-08-26T20:40:00+08:00"
    }
  ]
}
{
  "results": [
    {
      "location": {
        "id": "WX4FBXXFKE4F",
        "name": "北京",
        "country": "CN",
        "path": "北京,北京,中国",
        "timezone": "Asia/Shanghai",
        "timezone_offset": "+08:00"
      },
      "now": {
        "text": "阴",
        "code": "9",
        "temperature": "75",
        "fe

经过对比结果，我们发现心知天气API返回 摄氏度和华氏度 的数据的格式一样，转化为 dict 后，dict 的 keys 也不变，可以使用同样的下标取得需要的数据
接下来就开始实现：
- 解析 json 数据并根据当前温度单位输出天气信息的函数

In [10]:
"""
解析json数据
根据用户设定的温度显示单位输出天气信息
"""

def parse_weather_dict(result_dict_from_api, temperature_unit):
    """
    解析心知天气API返回的json格式数据，返回固定格式的天气信息
    :param result_dict_from_api: (dict), 天气 API 返回的 json 格式数据转换后的 dict
    :param temperature_unit: (str), 当前温度单位的辨识符.'c'代表摄氏度;'f'代表华氏度
    :return:(str), 当 API 返回正常时，返回天气信息；
             (None), 当 API 返回错误时，返回 None
    """

    # 判断API 返回数据是否正常
    # 如果正常，返回定制的天气信息
    # 确定天气单位描述文字
    if temperature_unit in ['c', 'C']:
        temperature_unit_text = '摄氏度'
    else:
        temperature_unit_text = '华氏度'

    # 如果返回 json 数据内有 results，说明获得了有效数据
    if result_dict_from_api.get('results', None) is not None:
        # 将数据拆分为 3 部分
        # location_dict: 存有天气对应地理位置的字典
        # updated_time_raw: 所获原始格式的天气信息最后更新时间
        # details_dict: 存有该地理位置的详细天气信息的字典，包括晴雨/温度等，需根据模式进行区分
        location_dict = result_dict_from_api['results'][0]['location']
        updated_time_raw = result_dict_from_api['results'][0]['last_update']
        updated_time_print = updated_time_raw.replace('T', ' ').replace('+08:00', '')

        now_details_dict = result_dict_from_api['results'][0]['now']
        result_for_user = dedent(f"""\
                {location_dict['name']}的实时天气为{now_details_dict['text']}
                气温: {now_details_dict['temperature']}{temperature_unit_text}
                风向：{now_details_dict['wind_direction']}
                更新时间:{updated_time_print}
        """)
        print(result_for_user)
        return result_for_user

    # 如果返回 json 数据内有 status，则异常
    elif result_dict_from_api.get('status', None):
        print('API返回异常,异常信息:', result_dict_from_api.get('status'))
        return None
    else:
        print('API返回异常，异常信息:', result_dict_from_api)
        return None
    
    
# 我们测试下这个函数能否正确完成我们的需求
# 调用 API

location = 'beijing'
# 调用 API 获得结果并转为 dict
result_dict_from_c = json.loads(fetchWeather(location, temperature_unit='c'))
result_dict_from_f = json.loads(fetchWeather(location, temperature_unit='f'))

# 解析 JSON 数据并输出天气信息
weather_search_result_c = parse_weather_dict(result_dict_from_c, 'c')
weather_search_result_f = parse_weather_dict(result_dict_from_f, 'f')


北京的实时天气为阴
气温: 24摄氏度
风向：西南
更新时间:2017-08-26 20:50:00

北京的实时天气为阴
气温: 75华氏度
风向：西南
更新时间:2017-08-26 20:55:00



至此，我们就完成了温度转换功能的开发，放在主函数中适当位置即可

### 1 查询指定日期的天气
这是 ch2 进阶任务中的一个，之前我们完成的程序仅能查询指定城市的实时天气，不能指定日期查询。在阅读心知天气 API 文档时，发现除了我们之前用的API URL，还有提供了另一个 API URL 可以查询未来15天内的天气概况。调用时的参数为：
- API URL：https://api.seniverse.com/v3/weather/daily.json
- api_params_dict： 跟之前比增加了2个 key: start 和 days。
    - start：值是阿拉伯数字，表示从哪天开始查询天气。0代表今天，1 代表明天，-1代表昨天，但免费用户不能查询历史天气信息
    - days: 值是阿拉伯数字，表示要查询的天数，从 start 那天开始算起。 1代表只查询 start 那天的天气

**难点**：

有了API和之前的探索基础，我们可以基于原来的框架和流程进行修改，完成功能开发，但是有一个问题：
- 我们之前的设计中，调用API是一个函数，解析数据并输出天气也是一个函数，如今需要换API，其返回的数据格式也会有变化，如何让这两个函数能兼容两种模式


**解决方案**：

经过思考，类似温度转换一样，引入一个变量解决以上难题：
- 使用一个变量`query_mode`标识当前的查询模式是 实时查询还是指定日期查询，它可以：
    - 由用户控制，经用户输入指令'now', 'future'完成改变
    - 在调用 API 时，根据它判断选择对应模式的那套 API 参数
    - 解析数据和输出天气信息时，根据它判断获得的数据是哪种模式，然后将数据套入对应模式的那套天气信息模板中，最后输出
- 在用户输入城市后，程序根据`query_mode`判断模式
    - 若为`now`, 进入实时查询流程，按我们之前的设计流程进行调用API，解析数据，输出天气信息
    - 若为`future`, 进入指定日期查询流程，要求用户继续输入，来确定`api_params_dict` 中的`start`和`days`的值，然后调用API，解析数据，输出天气
    





In [11]:
"""
引入变量，根据用户输入更改查询模式
"""
# 提示语
mode_switch_prompt = {
    'now': '您已经切换为查询：即时天气',
    'future':'您已经切换为查询：单日/多日天气概况',
}


query_mode = 'now'
while 1:
    user_input = input('请输入城市名称或指令，输入h/help获取帮助信息:')

    if user_input in ['now', 'future']:
        query_mode = user_input
        print(mode_switch_prompt[user_input])
    elif user_input in ['q', 'quit']:
        break


请输入城市名称或指令，输入h/help获取帮助信息:now
您已经切换为查询：即时天气
请输入城市名称或指令，输入h/help获取帮助信息:future
您已经切换为查询：单日/多日天气概况
请输入城市名称或指令，输入h/help获取帮助信息:q


In [7]:
"""
修改调用 API 拉取数据的函数，可以根据 query_mode 选择对应模式的那套 API 参数
"""

def fetch_weather(location=None, temperature_unit=None, query_mode=None):
    """
    调用心知天气查询 API
    :param location: (str), 查询天气时使用的城市名
    :param temperature_unit: (str), 查询天气时使用的温度单位
    :param query_mode: (str), 当前程序的查询模式，根据它选择对应的 API URL
    :return: (dict), 从 API 获得的天气数据
    """
    api_params_dict = dict(
            key='4r9bergjetiv1tsd', # API key
            location=location, # 查询的城市
            unit=temperature_unit,  # 单位 c 是摄氏度，f 是 华氏度
            language='zh-Hans',  # 查询结果的返回语言
)
    if query_mode == 'now':
        api_url = 'https://api.seniverse.com/v3/weather/now.json'
    else:
        api_url = 'https://api.seniverse.com/v3/weather/daily.json'
        start, days = get_query_date('   >>请输入查询的起始日期和天数，输入h获得帮助:') # 根据这里的调用方式，该函数在下面进行定义
        api_params_dict['start'] = start
        api_params_dict['days'] = days
        
    try:
        result = requests.get(api_url, params=api_params_dict, timeout=10)
    except requests.exceptions.ProxyError:
        print('网络不通，是不是网络断了？')
        return None
    except requests.exceptions.Timeout:
        print('查询天气的请求超过了5s仍未得到响应，网络可能不太好，请稍后再试')
        return None
    except:
        print('拉取天气数据时发生未知异常')
        return None

    else:
        # 解析 json 数据，获得 dict 格式数据
        # 注意这里之前的写法是 result.text
        # result.text 返回的是 str 格式的数据
        # result.json() 返回是 json 转化过的 python 内置数据结构，针对本api是dict
        result_dict_from_api = result.json()
        return result_dict_from_api

    
def get_query_date(prompt):
    """
    要求获得用户输入查询天气的起止日期
    :param prompt:(str), 提示语
    :return: (tuple), 起止日期
    """
    help_doc = dedent("""\
            请确定需要查询天气的起始日期和天数，最多只能查到未来15天内的天气
            -----------------输入说明及格式-----------------
            只能输入数字和英文逗号，格式为：0,1
            逗号前数字代表起始日期（0~14），如：0表示今天，1表示明天，以此类推
            逗号后数字代表从起始日期算起的天数(范围1~15)，1表示1天
            -----------------示例-----------------
            如查询今天天气，请输入：0,1
            如查询从今天起3日内天气，请输入：0,3
            如查询从明天起3日内天气，请输入：1,3
            """)
    error_msg = '输入错误，请重新输入。输入h可获得帮助信息'  
    while 1:
        try:
            user_input = input(prompt)
        except EOFError:
            print('您选择了退出交互。')
            exit(0)
        else:
            if user_input == 'h':
                print(help_doc)

            elif ',' in user_input:
                start, days = user_input.replace(' ','').split(',', 1)

                if start.isdigit() and days.isdigit():
                    return start, days
                else:
                    print(error_msg)

            else:
                print(error_msg)

In [8]:
"""
测试刚刚写的 调用API 的函数，看是否能正确返回数据
同时分析下，指定日期查询时，返回的数据格式，便于接下来写下一个函数
"""
import requests
import json

# 提示语
mode_switch_prompt = {
    'now': '您已经切换为查询：即时天气',
    'future':'您已经切换为查询：单日/多日天气概况',
}

query_mode = 'now'
temperature_unit = 'c'

for i in range(5):    # 在jupyter测试不用while 1 因为容易陷入循环中无法执行运行其他 cell 的代码
    user_input = input('请输入城市名称或指令，输入h/help获取帮助信息:')
    
    if user_input in ['now', 'future']:
        query_mode = user_input
        print(mode_switch_prompt[user_input])
    elif user_input in ['q', 'quit']:
        break

    elif is_city(user_input):
        result_dict_from_api = fetch_weather(location=user_input,
                                             temperature_unit=temperature_unit,
                                             query_mode=query_mode
                                             )
        format_result = json.dumps(result_dict_from_api, indent=2, ensure_ascii=False)
        print(format_result)

请输入城市名称或指令，输入h/help获取帮助信息:future
您已经切换为查询：单日/多日天气概况
请输入城市名称或指令，输入h/help获取帮助信息:cq
   >>请输入查询的起始日期和天数，输入h获得帮助:0,2
{
  "results": [
    {
      "location": {
        "id": "TVS86P4BSG4G",
        "name": "措勤",
        "country": "CN",
        "path": "措勤,阿里,西藏,中国",
        "timezone": "Asia/Shanghai",
        "timezone_offset": "+08:00"
      },
      "daily": [
        {
          "date": "2017-08-26",
          "text_day": "阴",
          "code_day": "9",
          "text_night": "多云",
          "code_night": "4",
          "high": "17",
          "low": "8",
          "precip": "",
          "wind_direction": "无持续风向",
          "wind_direction_degree": "0",
          "wind_speed": "10",
          "wind_scale": "2"
        },
        {
          "date": "2017-08-27",
          "text_day": "阵雨",
          "code_day": "10",
          "text_night": "小雨",
          "code_night": "13",
          "high": "16",
          "low": "8",
          "precip": "",
          "wind_direction": "无持续风向",
  

根据以上结果，我们就可以开始写解析数据并输出天气信息的函数啦

In [9]:
"""
可以根据 query_mode 识别当前模式，然后解析数据并输出天气信息
"""

def parse_weather_dict(result_dict_from_api=None, temperature_unit=None, query_mode=None):
    """
    解析心知天气API返回的json格式数据，返回固定格式的天气信息
    :param result_dict_from_api: (dict), 天气 API 返回的 json 格式数据转换后的 dict
    :param temperature_unit: (str), 当前温度单位的辨识符.'c'/ 'C'代表摄氏度;'f'/ 'F'代表华氏度
    :param query_mode: (str), 当前天气查询模式的辨识符。 'now'代表实时模式; 'daily'代表单日/多日模式
    :return:(str), 当 API 返回正常时，返回天气信息；
             (None), 当 API 返回错误时，返回 None
    """

    # 判断API 返回数据是否正常
    # 如果正常，返回定制的天气信息
    # 确定天气单位描述文字
    if temperature_unit in ['c', 'C']:
        temperature_unit_text = '摄氏度'
    else:
        temperature_unit_text = '华氏度'

    # 如果返回 json 数据内有 results，说明获得了有效数据
    if result_dict_from_api.get('results', None) is not None:
        # 将数据拆分为 3 部分
        # location_dict: 存有天气对应地理位置的字典
        # updated_time_raw: 所获原始格式的天气信息最后更新时间
        # details_dict: 存有该地理位置的详细天气信息的字典，包括晴雨/温度等，需根据模式进行区分
        location_dict = result_dict_from_api['results'][0]['location']
        updated_time_raw = result_dict_from_api['results'][0]['last_update']
        updated_time_print = updated_time_raw.replace('T', ' ').replace('+08:00', '')

        # 定制输出需要的天气信息
        if query_mode == 'now':  # 查询实时天气
            now_details_dict = result_dict_from_api['results'][0]['now']
            result_for_user = dedent(f"""\
                    {location_dict['name']}的实时天气为{now_details_dict['text']}
                    气温: {now_details_dict['temperature']}{temperature_unit_text}
                    风向：{now_details_dict['wind_direction']}
                    更新时间:{updated_time_print}
            """)
            print(result_for_user)
            return result_for_user

        else:  # 查询的是多日天气
            days_details_dict = result_dict_from_api['results'][0]['daily']
            result_for_user = f"{location_dict['name']}的天气为:\n"
            for day in days_details_dict:
                daily_result = dedent(f"""\
                    -----{day['date']}-----
                    白天：{day['text_day']}
                    夜间：{day['text_night']}
                    气温: {day['low']}~{day['high']}{temperature_unit_text}
                    风向：{day['wind_direction']}\n
                """)
                result_for_user += daily_result
            result_for_user += f"以上天气信息最后更新时间:{updated_time_print}"
            print(result_for_user)
            return result_for_user

    # 如果返回 json 数据内有 status，则异常
    elif result_dict_from_api.get('status', None):
        print('API返回异常,异常信息:', result_dict_from_api.get('status'))
        return None
    else:
        print('API返回异常，异常信息:', result_dict_from_api)
        return None

"""
测试以上函数是否正常完成我们的需求
"""
query_mode = 'now'
temperature_unit = 'c'

for i in range(5):    # 在jupyter测试不用while 1 因为容易陷入循环中无法执行运行其他 cell 的代码
    user_input = input('请输入城市名称或指令，输入h/help获取帮助信息:')
    
    if user_input in ['now', 'future']:
        query_mode = user_input
        print(mode_switch_prompt[user_input])
    elif user_input in ['q', 'quit']:
        break

    elif is_city(user_input):
        result_dict = fetch_weather(location=user_input,
                                             temperature_unit=temperature_unit,
                                             query_mode=query_mode
                                             )
        parse_weather_dict(result_dict_from_api=result_dict,
                           temperature_unit=temperature_unit,
                           query_mode=query_mode
                           )

请输入城市名称或指令，输入h/help获取帮助信息:cq
措勤的实时天气为多云
气温: 12摄氏度
风向：北
更新时间:2017-08-26 22:00:00

请输入城市名称或指令，输入h/help获取帮助信息:future
您已经切换为查询：单日/多日天气概况
请输入城市名称或指令，输入h/help获取帮助信息:cq
   >>请输入查询的起始日期和天数，输入h获得帮助:0,2
措勤的天气为:
-----2017-08-26-----
白天：阴
夜间：多云
气温: 8~17摄氏度
风向：无持续风向

-----2017-08-27-----
白天：阵雨
夜间：小雨
气温: 8~16摄氏度
风向：无持续风向

以上天气信息最后更新时间:2017-08-26 18:00:00
请输入城市名称或指令，输入h/help获取帮助信息:q


经过以上测试，我们完成了**指定日期查询天气**功能的全部开发

### 2 用户输入时自动补全

- **0 激发**：我给sunwenbo 学友 comment 中提到了猜测用户输入的城市并返回可能的城市列表，scottming 教练在看到后推荐命令行自动补全更人性化，并**再次**推荐Pycon2017 的演讲 Awesome Command Line Tools
    - 探索：
        - [Pycon演讲介绍](https://us.pycon.org/2017/schedule/presentation/518/)
        - [Pycon演讲视频](https://www.youtube.com/watch?v=hJhZhLg3obk)，28min的视频(后10min左右可学会设计一个自动补全的命令行程序)
    - 收获：
    	- 知道了如何用 `prompt-toolkits`这个包给命令行程序增加自动补全功能
    		- How to make a interactive program?
                - Read
                - Eval
                - Print
                - Loop
            - Check List
                - Persistent history
                - History search
                - Emacs Keybingdings
                - ~~Paged Output~~ 视频中未实现，但给出实现方案所需的库
                - Auto-Completion
                - Minimal Config
                - Syntax Coloring

    	- 第一次听说设计 CLI 程序遵守一定的原则，会让你的程序变得更棒
    		- **Discoverablity**  让程序的一些特色功能更容易被人发现，比如代码补全不用tab，在输入时就会提示
			- **User focus** 永远 user 第一，如何实现才是第二
			- **Configurablity** 如不需要，尽量减少配置文件，因为它的存在说明你不能准确把握什么样的配置对用户最佳（除了一些如色彩搭配的个人喜欢配置文件）

- **1 方案**
    - 根据视频推荐的第三方包 `prompt_toolkit` 可在用户输入时自动提示补全
        - 输入历史
        - 指定词汇
    - DEMO
        - 见下方代码

In [13]:
from prompt_toolkit import prompt
from prompt_toolkit.history import FileHistory
from prompt_toolkit.auto_suggest import AutoSuggestFromHistory
from prompt_toolkit.contrib.completers import WordCompleter

# 定义你需要自动补全的词汇表
city_list = ['北京', '天津', '南京', '重庆']

command_list = ['h', 'help',
                'q', 'quit',
                'history',
                'now', 'future',
                'c', 'f'
                ]
words_list = command_list + city_list



weather_completer = WordCompleter(words_list, ignore_case=False)

# 定义可自动补全的输入函数,用来代替内置的 input 函数
def prompt_input(prt):
    """
    定义有自动补全功能的函数获取用户输入并返回
    :param prt: 提示语
    :return: 用户输入
    """

    inp = prompt(prt,
                 history=FileHistory('history.txt'),
                 auto_suggest=AutoSuggestFromHistory(),
                 completer=weather_completer)
    return inp

# 在 jupyter 无法测试，需要拷贝至终端进行
# 经测试，没有问题，将该功能补充在程序中即可
for i in range(5):
    user_input = prompt_input(' >请输入城市名或指令:')

AssertionError: 

### 3 从嵌套的 JSON 数据中优雅得取数据
- **激发**
    - 自己和一些同学取值的办法即 json -> dict -> dict[key1][key2]...[keyn]
        - 这种办法把 key 写死在 code 中，一来若 api 接口改变,code也要改，麻烦；二来丑陋
    - 看到 [Vwan](https://github.com/AIHackers/Py101-004/issues/66#issuecomment-324340413)介绍自己使用了一种方法可以解决以上问题，然后就去其第一版作业中找到代码并阅读。
    - 读完之后觉得十分精妙，十分精妙！Vwan 使用类似 `.attr`的方式从json中取得数据，而且把`dict.key1.key2...keyn`这种嵌套结构放在一个配置文件j下，方便改变。不过该方法有个缺点是只能处理json中无数组的情况，后来Vwan又对该方法进行了改进，使得可以解决json内嵌套数组的情况，其本人的[学习笔记](https://github.com/Vwan/Py101-004/blob/master/Chap2/note/Exercise%20Notes%20-%20chap2.md)内有讲解
- **探索**
    - 看完第一版代码后，自己clone下代码在本地进行测试，加深理解，DEMO见下方
    - 准备自己思考一下如何处理数组的情况，然后与Vwan方案进行对比
- **总结**
    - 人外有人，第一次近距离感觉到“开源”的魅力，不用重复造轮子，而且即使造了，也不一定有别人的好，但思考过程能帮助人成长

In [14]:
def parse_json(json_data, **required_data):
    extracted_data = {}
    for data,key in required_data.items():
        tmp = json_data
        if ("." in data):
            temp_keys = data.split(".")
            for temp_key in temp_keys:
                tmp = tmp[temp_key]
            extracted_data[key] = tmp
        else:
            extracted_data[key] = json_data[data]
    return extracted_data

json_data = {
  "HeWeather5": [
    {
      "basic": {
        "city": "北京",
        "cnty": "中国",
        "id": "CN101010100",
        "lat": "39.90498734",
        "lon": "116.40528870",
        "update": {
          "loc": "2017-08-23 23:46",
          "utc": "2017-08-23 15:46"
        }
      },
      "now": {
        "cond": {
          "code": "100",
          "txt": "晴"
        },
        "fl": "28",
        "hum": "73",
        "pcpn": "0",
        "pres": "1007",
        "tmp": "26",
        "vis": "7",
        "wind": {
          "deg": "275",
          "dir": "西风",
          "sc": "微风",
          "spd": "6"
        }
      },
      "status": "ok"
    }
  ]
}

required_data = {
  'basic.city': 'City',
  'now.cond.txt': 'Weather',
  'now.wind.dir': "WInd",
  'now.tmp': "Temperature",
  'basic.update.utc': 'Last Updated On',
}

# 使用方法
extracted_data = parse_json(json_data['HeWeather5'][0],**required_data)
print(extracted_data)

{'City': '北京', 'Weather': '晴', 'WInd': '西风', 'Temperature': '26', 'Last Updated On': '2017-08-23 15:46'}


## Timelog

- 构思框架 5min
- 撰写 Draft 20min
- 撰写 Action 60min
    - 了解 API
    - 了解 JSON
- 进阶功能
    - 温度单位转换 40min
    - 指定日期查询天气 80min
    - 输入自动补全 15min
    - 从嵌套的 JSON 数据中优雅得取数据 20min